In [5]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [6]:
filelist = glob.glob('train/final_glu_aug/*.jpg')              #change '/'
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(357, 605, 700, 3)


In [ ]:
filelist = glob.glob('train/final_non_glu_aug/*.jpg')         #change '/'
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

In [ ]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)

In [ ]:
gluL = np.array([1 for i in range(357)])
nongluL = np.array([0 for i in range(357)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense
model = Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), strides =2))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.output_shape

In [ ]:
model.summary()

In [ ]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2, batch_size=1,validation_data=(X_test, y_test))

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=1)
print(score)

In [ ]:
model.predict(X_test, batch_size=1)
model.predict_classes(X_test,batch_size=1)

In [ ]:
import h5py
model.save_weights('weights.h5')